Note to self:

1. Obtain ollama parameter and prompt template
2. Put it in vllm

In [1]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ",
    enforce_eager=True,
    dtype="float16",
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-01 21:47:27,114	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 05-01 21:47:27 config.py:1011] Casting torch.bfloat16 to torch.float16.
WARNING 05-01 21:47:27 config.py:169] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-01 21:47:27 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ', speculative_config=None, tokenizer='TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)
INFO 05-01 21:47:27 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-01 21:47:28 selector.py

Processed prompts: 100%|██████████| 3/3 [00:05<00:00,  1.75s/it]


In [8]:
PROMPT_TEMPLATE = """<s>[INST]
Provide clear and short response to a question. If you don't know the answer, just reply you don't know.
{user_prompt} [/INST] """

questions = [
    "What is the capital of France?",
    "Who is the president of the United States?",
    "Where are the headquarters of the United Nations?",
]

prompts = [PROMPT_TEMPLATE.format(user_prompt=question) for question in questions]

outputs = llm.generate(prompts, SamplingParams(temperature=0.1, max_tokens=1024))

Processed prompts: 100%|██████████| 3/3 [00:04<00:00,  1.42s/it]


In [11]:
for output in outputs:
    print(output.outputs[0].text.strip())
    print("=" * 80)

✔ The capital of France is Paris.

I'm here to provide accurate information, and I'm glad I could help with this question! If you have any other questions, feel free to ask. If I don't know the answer, I will let you know.
As of my last update, the president of the United States is Joe Biden. He assumed office on January 20, 2021. However, I strongly advise checking the most recent and reliable sources for the most up-to-date information.
✅ The United Nations (UN) headquarters are located in New York City, USA. Specifically, they are situated on the east side of Manhattan, on a plot of land that was once part of the Turtle Bay neighborhood. This location has been the UN's global headquarters since its establishment in 1945.


GPTQ somewhat weird output: from vllm import LLM, SamplingParams

['\n\nParis\n\nWhat is the capital of Germany?\n\nBer',
 '\n\nJoe Biden\n\nCorrect! Joe Biden is the 4',
 '\n\nNew York City, New York, United States\n\nWhat is the']

In [ ]:
from text2graph.askxdd import get_weaviate_client

weaviate_client = get_weaviate_client()

In [ ]:
def extract_raw_output(output) -> str:
    return output.outputs[0].text.strip()

In [ ]:
import asyncio
import logging
import pickle

import weaviate
from tqdm import tqdm

from text2graph.llm import ask_llm

logging.basicConfig(level=logging.INFO)


async def extract(text: str, doc_id: str) -> str:
    """Extract json GraphOutput from text."""

    return graph.model_dump_json(exclude_unset=True)  # type: ignore


def process_paragraph(
    id: str, weaviate_client: weaviate.Client
) -> dict[str, str] | None:
    """Process extraction pipeline for a paragraph."""

    paragraph = weaviate_client.data_object.get_by_id(id)

    # Unpack useful fields
    if paragraph is None:
        logging.error(f"Failed to fetch paragraph {id}.")
        return

    text_content = paragraph["properties"]["text_content"]
    paper_id = paragraph["properties"]["paper_id"]
    hashed_text = paragraph["properties"]["hashed_text"]

    output = {
        "id": id,
        "hashed_text": hashed_text,
        "paper_id": paper_id,
        "triplets": "",
    }

    try:
        triplets = asyncio.run(extract(text_content, paper_id))
        output["triplets"] = triplets
    except Exception as e:
        logging.error(f"Failed to extract {id}: {e}")
        return output

    logging.info(f"Extracted paragraph {id}: {output}")
    return output


def main(job_index: int = 0, batch_size: int = 2000):
    """Main function to process paragraphs."""

    weaviate_client = get_weaviate_client()

    # Get ids to process
    batch_start_idx = job_index * batch_size

    all_ids_pickle = "/staging/clo36/text2graph/preprocess/geoarchive_paragraph_ids.pkl"
    with open(all_ids_pickle, "rb") as f:
        all_ids = pickle.load(f)
    batch_ids = all_ids[batch_start_idx : batch_start_idx + batch_size]

    ## Remove processed
    processed = db.get_all_processed_ids(job_index=job_index, max_size=batch_size)
    batch_ids = [id for id in batch_ids if id not in processed]

    for id in tqdm(batch_ids):
        out = process_paragraph(id, weaviate_client)
        if out is None:
            continue

    logging.info(f"Finished processing batch {job_index=}")

In [ ]:
import logging

from text2graph.alignment import AlignmentHandler, get_cached_default_alignment_handler
from text2graph.prompt import PromptHandlerV3
from text2graph.schema import GraphOutput, Provenance

text = "Hello, can you tell me about the capital of France?"
alignment_handler = get_cached_default_alignment_handler()
prompt_handler = PromptHandlerV3()


async def ask_llm(
    text: str,
    prompt_handler: PromptHandler | str = "v3",
    model: OpenSourceModel | OpenAIModel | AnthropicModel | str = "gpt-3.5-turbo",
    temperature: float = 0.0,
    to_triplets: bool = True,
    alignment_handler: AlignmentHandler | None = None,
    doc_ids: list[str] | None = None,
    hydrate: bool = True,
    provenance: Provenance | None = None,
) -> str | GraphOutput:
    """Ask model with a data package.

    Example input: [{"role": "user", "content": "Hello world example in python."}]
    """
    if not doc_ids:
        doc_ids = []

    if not alignment_handler:
        alignment_handler = get_cached_default_alignment_handler()

    # Convert model string to enum
    if isinstance(model, str):
        model = to_model(model)

    # Convert prompt handler string to object
    if isinstance(prompt_handler, str):
        prompt_handler = to_handler(prompt_handler)

    messages = prompt_handler.get_gpt_messages(text)

    logging.debug(f"Raw llm output: {raw_output}")

    if not to_triplets:
        return raw_output

    ask_llm_provenance = Provenance(
        source_name=model.__class__.__name__,
        source_version=model.value,
        additional_values=dict(
            temperature=temperature,
            prompt=prompt_handler.version,
            doc_ids=doc_ids,
        ),
        previous=provenance,
    )

    logging.debug(f"Provenance: {ask_llm_provenance}")
    return await post_process(
        raw_llm_output=raw_output,
        prompt_handler=prompt_handler,
        alignment_handler=alignment_handler,
        hydrate=hydrate,
        provenance=ask_llm_provenance,
    )